In [12]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from tabulate import tabulate
from collections import Counter
import nltk
nltk.download('punkt')
import string

[nltk_data] Downloading package punkt to C:\Users\Nur
[nltk_data]     Adilah\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
#import pandas module for creating dataframe
import pandas as pd

#read CSV into DataFrame
dataset = pd.read_csv("customer_complaints_1.csv")
dataset

,author,posted_on,rating,text
0,"Alantae of Chesterfeild, MI","Nov. 22, 2016",1,I used to love Comcast. Until all these consta...
1,"Vera of Philadelphia, PA","Nov. 19, 2016",1,I'm so over Comcast! The worst internet provid...
2,"Sarah of Rancho Cordova, CA","Nov. 17, 2016",1,If I could give them a negative star or no sta...
3,"Dennis of Manchester, NH","Nov. 16, 2016",1,I've had the worst experiences so far since in...
4,"Ryan of Bellevue, WA","Nov. 14, 2016",1,Check your contract when you sign up for Comca...
5,"Terri of Mobile, AL","Nov. 9, 2016",1,Thank God. I am changing to Dish. They gave me...
6,"Kellie of Salt Lake City, UT","Nov. 9, 2016",1,I Have been a long time customer and only have...
7,"Kathleen of New Haven, CT","Nov. 6, 2016",2,There is a malfunction on the DVR manager whic...
8,"Shira of Bloomfield, NJ","Nov. 5, 2016",1,Charges overwhelming. Comcast service rep was ...
9,"Kristy of Alpharetta, GA","Nov. 2, 2016",1,"I have had cable, DISH, and U-verse, etc. in t..."


In [14]:
dataset.drop(dataset.columns[[0, 1, 2]], axis=1, inplace=True)
dataset

,text
0,I used to love Comcast. Until all these consta...
1,I'm so over Comcast! The worst internet provid...
2,If I could give them a negative star or no sta...
3,I've had the worst experiences so far since in...
4,Check your contract when you sign up for Comca...
5,Thank God. I am changing to Dish. They gave me...
6,I Have been a long time customer and only have...
7,There is a malfunction on the DVR manager whic...
8,Charges overwhelming. Comcast service rep was ...
9,"I have had cable, DISH, and U-verse, etc. in t..."


In [15]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
# Define the function to remove punctuation
def remove_punctuation(text):
    # Import the string module
    import string
    
    # Initialize an empty string to store the result
    punctuation_free = ""
    
    # Iterate over each character in the text
    for char in text:
        # Check if the character is not a punctuation
        if char not in string.punctuation:
            # If not, add the character to the result string
            punctuation_free += char
    
    return punctuation_free

dataset = list(map(remove_punctuation, dataset['text']))
dataset

['I used to love Comcast Until all these constant updates My internet and cable crash a lot at night and sometimes during the day some channels dont even work and on demand sometimes dont play either I wish they will do something about it Because just a few mins ago the internet have crashed for about 20 mins for no reason Im tired of it and thinking about switching to Wow or something Please do not get Xfinity',
 'Im so over Comcast The worst internet provider Im taking online classes and multiple times was late with my assignments because of the power interruptions in my area that lead to poor quality internet service Definitely switching to Verizon Id rather pay 10 extra then dealing w Comcast and non stopping internet problems',
 'If I could give them a negative star or no stars on this review I would I have never worked with any industry with as bad of customer service as Comcast It is not a matter of money because I make well enough above and beyond to afford their services but t

In [19]:
#to standardize the cases in the documents into lower case
dataset = list(map(str.lower,dataset))

dataset

['i used to love comcast until all these constant updates my internet and cable crash a lot at night and sometimes during the day some channels dont even work and on demand sometimes dont play either i wish they will do something about it because just a few mins ago the internet have crashed for about 20 mins for no reason im tired of it and thinking about switching to wow or something please do not get xfinity',
 'im so over comcast the worst internet provider im taking online classes and multiple times was late with my assignments because of the power interruptions in my area that lead to poor quality internet service definitely switching to verizon id rather pay 10 extra then dealing w comcast and non stopping internet problems',
 'if i could give them a negative star or no stars on this review i would i have never worked with any industry with as bad of customer service as comcast it is not a matter of money because i make well enough above and beyond to afford their services but t

In [20]:
## Remove numbers using re.sub ( ) in regular expression library

#import regular expression library
import re

#function to remove digit (\d) or hypens (-) from the documents with an empty string ''
def remove_numbers(text):
    return re.sub("[\d-]",'',text)

#applying the remove_numbers function to the 'clean_lower' column and storing the result in a new column 'clean_number'
dataset = list(map(remove_numbers,dataset))

dataset


['i used to love comcast until all these constant updates my internet and cable crash a lot at night and sometimes during the day some channels dont even work and on demand sometimes dont play either i wish they will do something about it because just a few mins ago the internet have crashed for about  mins for no reason im tired of it and thinking about switching to wow or something please do not get xfinity',
 'im so over comcast the worst internet provider im taking online classes and multiple times was late with my assignments because of the power interruptions in my area that lead to poor quality internet service definitely switching to verizon id rather pay  extra then dealing w comcast and non stopping internet problems',
 'if i could give them a negative star or no stars on this review i would i have never worked with any industry with as bad of customer service as comcast it is not a matter of money because i make well enough above and beyond to afford their services but they 

In [21]:
tokenized_dataset = [doc.split() for doc in dataset]
tokenized_dataset


[['i',
  'used',
  'to',
  'love',
  'comcast',
  'until',
  'all',
  'these',
  'constant',
  'updates',
  'my',
  'internet',
  'and',
  'cable',
  'crash',
  'a',
  'lot',
  'at',
  'night',
  'and',
  'sometimes',
  'during',
  'the',
  'day',
  'some',
  'channels',
  'dont',
  'even',
  'work',
  'and',
  'on',
  'demand',
  'sometimes',
  'dont',
  'play',
  'either',
  'i',
  'wish',
  'they',
  'will',
  'do',
  'something',
  'about',
  'it',
  'because',
  'just',
  'a',
  'few',
  'mins',
  'ago',
  'the',
  'internet',
  'have',
  'crashed',
  'for',
  'about',
  'mins',
  'for',
  'no',
  'reason',
  'im',
  'tired',
  'of',
  'it',
  'and',
  'thinking',
  'about',
  'switching',
  'to',
  'wow',
  'or',
  'something',
  'please',
  'do',
  'not',
  'get',
  'xfinity'],
 ['im',
  'so',
  'over',
  'comcast',
  'the',
  'worst',
  'internet',
  'provider',
  'im',
  'taking',
  'online',
  'classes',
  'and',
  'multiple',
  'times',
  'was',
  'late',
  'with',
  'my',
 

In [22]:
#download stopwords
nltk.download('stopwords')

#Get the list of English stop words present in the library 
stopwords = nltk.corpus.stopwords.words('english')

stopwords


[nltk_data] Downloading package stopwords to C:\Users\Nur
[nltk_data]     Adilah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [23]:
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output = []
    for i in text:
        if i not in stopwords:
            output.append(i)
    return output

#Applying the remove_stopwords function to the 'token_data' column and storing the result in a new column 'clean_xstopwords'
tokenized_dataset = list(map(remove_stopwords,tokenized_dataset))
tokenized_dataset


[['used',
  'love',
  'comcast',
  'constant',
  'updates',
  'internet',
  'cable',
  'crash',
  'lot',
  'night',
  'sometimes',
  'day',
  'channels',
  'dont',
  'even',
  'work',
  'demand',
  'sometimes',
  'dont',
  'play',
  'either',
  'wish',
  'something',
  'mins',
  'ago',
  'internet',
  'crashed',
  'mins',
  'reason',
  'im',
  'tired',
  'thinking',
  'switching',
  'wow',
  'something',
  'please',
  'get',
  'xfinity'],
 ['im',
  'comcast',
  'worst',
  'internet',
  'provider',
  'im',
  'taking',
  'online',
  'classes',
  'multiple',
  'times',
  'late',
  'assignments',
  'power',
  'interruptions',
  'area',
  'lead',
  'poor',
  'quality',
  'internet',
  'service',
  'definitely',
  'switching',
  'verizon',
  'id',
  'rather',
  'pay',
  'extra',
  'dealing',
  'w',
  'comcast',
  'non',
  'stopping',
  'internet',
  'problems'],
 ['could',
  'give',
  'negative',
  'star',
  'stars',
  'review',
  'would',
  'never',
  'worked',
  'industry',
  'bad',
  'cus

In [24]:
## Perform word stemming using Lancaster Stemmer in nltk librarY

#importing the Stemming function from nltk library
from nltk.stem.lancaster import LancasterStemmer

#defining the object for stemming
lancaster_stemmer = LancasterStemmer()

#defining a function for stemming
def stemming(text):
    stem_text = []
    for word in text:
        stemmed_word = lancaster_stemmer.stem(word)
        stem_text.append(stemmed_word)
    return stem_text

#applying the stemming function to the 'clean_xstopwords' column and storing the result in a new column 'clean_stemmed' 
stemmed_dataset = list(map(stemming,tokenized_dataset))
stemmed_dataset


[['us',
  'lov',
  'comcast',
  'const',
  'upd',
  'internet',
  'cabl',
  'crash',
  'lot',
  'night',
  'sometim',
  'day',
  'channel',
  'dont',
  'ev',
  'work',
  'demand',
  'sometim',
  'dont',
  'play',
  'eith',
  'wish',
  'someth',
  'min',
  'ago',
  'internet',
  'crash',
  'min',
  'reason',
  'im',
  'tir',
  'think',
  'switch',
  'wow',
  'someth',
  'pleas',
  'get',
  'xfin'],
 ['im',
  'comcast',
  'worst',
  'internet',
  'provid',
  'im',
  'tak',
  'onlin',
  'class',
  'multipl',
  'tim',
  'lat',
  'assign',
  'pow',
  'interrupt',
  'are',
  'lead',
  'poor',
  'qual',
  'internet',
  'serv',
  'definit',
  'switch',
  'verizon',
  'id',
  'rath',
  'pay',
  'extr',
  'deal',
  'w',
  'comcast',
  'non',
  'stop',
  'internet',
  'problem'],
 ['could',
  'giv',
  'neg',
  'star',
  'star',
  'review',
  'would',
  'nev',
  'work',
  'industry',
  'bad',
  'custom',
  'serv',
  'comcast',
  'mat',
  'money',
  'mak',
  'wel',
  'enough',
  'beyond',
  'afford

In [25]:
## Perform word lemmatization using WordNetLemmatizer( ) in nltk library

nltk.download('wordnet')

#importing the Lemmatizer function from nltk library
from nltk.stem import WordNetLemmatizer

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = []
    for word in text:
        lemmatized_word = wordnet_lemmatizer.lemmatize(word)
        lemm_text.append(lemmatized_word)
    return lemm_text

# #applying the lemmatizer function to the 'clean_xstopwords' column and storing the result in a new column 'clean_lemmatized1'
dataset_lemm_stem = list(map(lemmatizer,stemmed_dataset))

# #applying the lemmatizer function to the 'clean_stemmed' column and storing the result in a new column 'clean_lemmatized2'
dataset_lemm = list(map(lemmatizer,tokenized_dataset))

print(dataset_lemm_stem)

print(dataset_lemm)


[nltk_data] Downloading package wordnet to C:\Users\Nur
[nltk_data]     Adilah\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[['u', 'lov', 'comcast', 'const', 'upd', 'internet', 'cabl', 'crash', 'lot', 'night', 'sometim', 'day', 'channel', 'dont', 'ev', 'work', 'demand', 'sometim', 'dont', 'play', 'eith', 'wish', 'someth', 'min', 'ago', 'internet', 'crash', 'min', 'reason', 'im', 'tir', 'think', 'switch', 'wow', 'someth', 'plea', 'get', 'xfin'], ['im', 'comcast', 'worst', 'internet', 'provid', 'im', 'tak', 'onlin', 'class', 'multipl', 'tim', 'lat', 'assign', 'pow', 'interrupt', 'are', 'lead', 'poor', 'qual', 'internet', 'serv', 'definit', 'switch', 'verizon', 'id', 'rath', 'pay', 'extr', 'deal', 'w', 'comcast', 'non', 'stop', 'internet', 'problem'], ['could', 'giv', 'neg', 'star', 'star', 'review', 'would', 'nev', 'work', 'industry', 'bad', 'custom', 'serv', 'comcast', 'mat', 'money', 'mak', 'wel', 'enough', 'beyond', 'afford', 'serv', 'legitim', 'ripoff', 'think', 'biggest', 'scam', 'sint', 'mortg', 'industry', 'maj', 'meltdown', 'hop', 'mov', 'somewh', 'comcast', 'ex', 'disregard', 'want', 'help', 'right',

In [26]:
word2vec_model = Word2Vec(sentences=dataset_lemm, vector_size=100, 
window=5, min_count=1, workers=4)


NameError: name 'Word2Vec' is not defined

In [27]:
# Assuming dataset_lemm_stem is a list of lists
dataset_lemm_concatenated = [' '.join(words) for words in dataset_lemm]

# Now, use the TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset_lemm_concatenated)

In [28]:
k = 2 # Define the number of clusters
km = KMeans(n_clusters=k)
km.fit(X)
# Predict the clusters for each document
y_pred = km.predict(X)
# Display the document and its predicted cluster in a table
table_data = [["Document", "Predicted Cluster"]]
table_data.extend([[doc, cluster] for doc, cluster in zip(dataset, y_pred)])
print(tabulate(table_data, headers="firstrow"))
# Print top terms per cluster
print("\nTop terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(k):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
    print()

C:\Users\Nur Adilah\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Document                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [29]:
# Calculate purity
total_samples = len(y_pred)
cluster_label_counts = [Counter(y_pred)]
purity = sum(max(cluster.values()) for cluster in cluster_label_counts) / total_samples
print("Purity:", purity)

Purity: 0.7894736842105263
